## Downloads
Clone Git repo
Get Athena JDBC driver

In [7]:
%%bash

git clone https://github.com/aws-samples/amazon-neptune-graph-virtualization.git
ls amazon-neptune-graph-virtualization

wget https://downloads.athena.us-east-1.amazonaws.com/drivers/JDBC/SimbaAthenaJDBC-2.1.0.1000/AthenaJDBC42-2.1.0.1000.jar
#wget https://downloads.athena.us-east-1.amazonaws.com/drivers/JDBC/SimbaAthenaJDBC-2.1.0.1000/SimbaAthenaJDBC-2.1.0.1000.zip
#wget https://s3.amazonaws.com/athena-downloads/drivers/JDBC/SimbaAthenaJDBC-2.0.33.1002/AthenaJDBC42-2.0.33.jar


fatal: destination path 'amazon-neptune-graph-virtualization' already exists and is not an empty directory.


cfn
CODE_OF_CONDUCT.md
CONTRIBUTING.md
LICENSE
notebook
ontop
README.md
scripts


--2023-08-02 16:34:39--  https://downloads.athena.us-east-1.amazonaws.com/drivers/JDBC/SimbaAthenaJDBC-2.1.0.1000/AthenaJDBC42-2.1.0.1000.jar
Resolving downloads.athena.us-east-1.amazonaws.com (downloads.athena.us-east-1.amazonaws.com)... 18.66.171.33, 18.66.171.103, 18.66.171.117, ...
Connecting to downloads.athena.us-east-1.amazonaws.com (downloads.athena.us-east-1.amazonaws.com)|18.66.171.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176 [application/java-archive]
Saving to: ‘AthenaJDBC42-2.1.0.1000.jar’

     0K                                                       100% 32.1M=0s

2023-08-02 16:34:39 (32.1 MB/s) - ‘AthenaJDBC42-2.1.0.1000.jar’ saved [176/176]



## Get info about this environment

In [4]:
import os
import subprocess

stream = os.popen("source ~/.bashrc ; echo $DATA_BUCKET; echo $PUB_BUCKET; echo $ROOT_DIR_NOSLASH; echo $STACKNAME")
lines=stream.read().split("\n")
DATA_BUCKET=lines[0]
PUB_BUCKET=lines[1]
ROOT_DIR_NOSLASH=lines[2]
STACKNAME=lines[3]
lines


['sligo6-s3databucket-k7ctmci9qc51',
 'aws-neptune-customer-samples',
 'neptune-virtualization/blog',
 '']

In [10]:
%%bash
REGION=`aws configure get region`
ACCOUNT=`aws sts get-caller-identity --query "Account" --output text`
echo $REGION
echo $ACCOUNT
echo "REGION " $REGION >> vars.txt
echo "ACCOUNT " $ACCOUNT >> vars.txt


eu-west-1
346782484055


In [ ]:
%%sparql
PREFIX clmo: <http://climate.aws.com/ontology/>
PREFIX clmr: <http://climate.aws.com/resource#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>


# readings at Manchester UK in 
select ?name ?dateTime ?fahrenheit ?celsius where {
    BIND(clmr:3334099999 as ?stat)
    ?stat rdfs:label ?name .
    
    SERVICE <http://44.214.5.75:8080/sparql> {
        ?reading clmo:station ?stat ;
            clmo:dateTime ?dateTime ;
            clmo:celsius ?celsius ;
            clmo:fahrenheit ?fahrenheit .
        FILTER(?dateTime > "1985-12-30T13:20:00"^^xsd:dateTime)
    } 
} 
order by ASC(?dateTime)


In [ ]:
%%sparql
PREFIX clmo: <http://climate.aws.com/ontology/>
PREFIX clmr: <http://climate.aws.com/resource#>


select * where {
    BIND(clmr:78259399999 as ?stat)
    SERVICE <http://44.214.5.75:8080/sparql> {
        clmr:78259399999 a ?type ;
    } 
} 



In [80]:
%load

Button(description='Submit', style=ButtonStyle())

Output()

In [65]:
%%sparql
PREFIX clmo: <http://climate.aws.com/ontology/>
PREFIX clmr: <http://climate.aws.com/resource#>


select * where {

    SERVICE <http://44.214.5.75:8080/sparql> {
        clmr:74063099999 a clmo:WeatherStation ;
        ?p ?o .
    } 
} 

In [82]:
%%sparql

select * where {
?s ?p ?o .
} limit 10